<a href="https://colab.research.google.com/github/psa2001/computer_vision/blob/main/bird_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd

/root


In [ ]:
cd ..

/


In [ ]:
import os
from pathlib import Path
dataset_dir = Path('content/drive/My Drive/bird_classifiaction')
os.chdir(dataset_dir)   

In [ ]:
"""
import zipfile
with zipfile.ZipFile('public_tests.zip', 'r') as zip_ref:
    zip_ref.extractall()
"""

"\nimport zipfile\nwith zipfile.ZipFile('public_tests.zip', 'r') as zip_ref:\n    zip_ref.extractall()\n"

In [ ]:
def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            coords = np.array([float(x) for x in parts[1:]], dtype='float64')
            res[parts[0]] = coords
    return res


In [ ]:
cd

/root


In [ ]:
cd ..

/


In [ ]:
dataset_dir = Path('content/drive/My Drive/bird_classifiaction/public_tests/00_test_img_input/train/')
os.chdir(dataset_dir)  


In [ ]:
import numpy as np
from PIL import Image

from tqdm import tqdm
import matplotlib.pyplot as plt



set_all_images = []
answ = []

answ_files = read_csv('gt.csv')
os.chdir('images/')

for f in tqdm (sorted(os.listdir())):
    
  jpg = Image.open(f)

  gt = answ_files[f]


  answ.append(gt) 
  
  fixed_size_jpg = jpg.resize((197, 197))
  
  np_image = np.array(fixed_size_jpg)

  if(np.size(np_image.shape) == 2):

    k = np.dstack((np_image, np_image, np_image))

    set_all_images.append(k)

  else:
    
    set_all_images.append(np_image)

100%|██████████| 2500/2500 [00:19<00:00, 127.83it/s]


In [ ]:
set_all_images = np.array(set_all_images, dtype = np.float64)
set_all_images /= 255.0

In [ ]:
set_all_images.shape

(2500, 197, 197, 3)

In [ ]:
answ = np.array(answ, dtype = np.float64)

In [ ]:
import tensorflow as tf

answ_cat = np.array(tf.keras.utils.to_categorical(answ, num_classes=50))
answ_cat.shape

(2500, 50)

In [ ]:
set_all_images.shape

(2500, 197, 197, 3)

#Моделька

In [ ]:
import tensorflow as tf

base_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(250, 250, 3),
    include_top=False)

base_model.trainable = False

inputs = tf.keras.Input(shape=(250, 250, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)

outputs =  tf.keras.layers.Dense(50, activation = 'softmax')(x)

model = tf.keras.Model(inputs, outputs)


In [ ]:
#base_model.summary()

In [ ]:
"""
for layer in base_model.layers[-5:]:
   layer.trainable = True
"""

'\nfor layer in base_model.layers[-5:]:\n   layer.trainable = True\n'

In [ ]:
#model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam


model.compile(
    loss = 'categorical_crossentropy',
    optimizer='rmsprop',
    metrics = ['mse', 'accuracy']
)


chkpt = tf.keras.callbacks.ModelCheckpoint("bird_classification.hdf5", 
                        monitor='val_loss', 
                        verbose=1, 
                        save_best_only=True, 
                        mode='auto')

In [ ]:
import tensorflow as tf

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,)

In [ ]:
def shuffle(x, y):
    p = np.random.permutation(len(x)).astype(int)
    x = x[p,]
    y = y[p,]
    return x, y

In [ ]:
images, gts = shuffle(set_all_images, answ_cat)

In [ ]:
x_vld = set_all_images[::10]
y_vld = answ_cat[::10]
x_trn = np.array([])
y_trn = np.array([])
for i in range(1,10):
  if (i == 1):
    x_trn = np.copy(set_all_images[i::10])
    y_trn = np.copy(answ_cat[i::10])
  else:
    x_trn = np.concatenate((x_trn, set_all_images[i::10]), axis = 0)
    y_trn = np.concatenate((y_trn, answ_cat[i::10]), axis = 0)

In [ ]:
x_trn.shape, y_trn.shape, x_vld.shape, y_vld.shape

((2250, 197, 197, 3), (2250, 50), (250, 197, 197, 3), (250, 50))

In [ ]:
 model.fit(datagen.flow(x_trn,
          y_trn,
          batch_size=120),
          epochs=100,
          verbose=1,
          validation_data = (x_vld, y_vld),
          shuffle=True)

Epoch 1/100
19/19 [==============================] - 23s 1s/step - loss: 2.8671 - mse: 0.0175 - accuracy: 0.2778 - val_loss: 3.8451 - val_mse: 0.0196 - val_accuracy: 0.4040
Epoch 2/100
19/19 [==============================] - 20s 1s/step - loss: 1.8988 - mse: 0.0141 - accuracy: 0.4502 - val_loss: 2.9428 - val_mse: 0.0175 - val_accuracy: 0.4520
Epoch 3/100
19/19 [==============================] - 20s 1s/step - loss: 1.7164 - mse: 0.0132 - accuracy: 0.4898 - val_loss: 2.6870 - val_mse: 0.0161 - val_accuracy: 0.4840
Epoch 4/100
19/19 [==============================] - 21s 1s/step - loss: 1.6046 - mse: 0.0127 - accuracy: 0.5191 - val_loss: 2.6457 - val_mse: 0.0167 - val_accuracy: 0.4760
Epoch 5/100
19/19 [==============================] - 21s 1s/step - loss: 1.4528 - mse: 0.0118 - accuracy: 0.5524 - val_loss: 2.2134 - val_mse: 0.0166 - val_accuracy: 0.4680
Epoch 6/100
19/19 [==============================] - 21s 1s/step - loss: 1.3782 - mse: 0.0111 - accuracy: 0.5871 - val_loss: 2.0838 - v

KeyboardInterrupt: ignored

In [ ]:
model.save('first_etap.hdf5')

In [ ]:
from keras.models import load_model

model = load_model('second_etap.hdf5')
model.evaluate(set_all_images, answ_cat)

79/79 [==============================] - 8s 99ms/step - loss: 0.1828 - mean_squared_error: 0.0014 - accuracy: 0.9516


[0.1827937662601471, 0.0014452259056270123, 0.9516000151634216]

In [ ]:
for layer in base_model.layers[-19:]:
   layer.trainable = True

In [ ]:
from keras import optimizers

model.compile(
    loss = 'categorical_crossentropy',
    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
    metrics = ['mse', 'accuracy']
)

In [ ]:
 model.fit(datagen.flow(x_trn,
          y_trn,
          batch_size=120),
          epochs=200,
          verbose=1,
          validation_data = (x_vld, y_vld),
          shuffle=True)

Epoch 1/200
19/19 [==============================] - 20s 1s/step - loss: 0.1704 - mse: 0.0017 - accuracy: 0.9387 - val_loss: 1.4525 - val_mse: 0.0103 - val_accuracy: 0.6520
Epoch 2/200
19/19 [==============================] - 20s 1s/step - loss: 0.1674 - mse: 0.0016 - accuracy: 0.9467 - val_loss: 1.4448 - val_mse: 0.0102 - val_accuracy: 0.6560
Epoch 3/200
19/19 [==============================] - 19s 1s/step - loss: 0.1766 - mse: 0.0017 - accuracy: 0.9467 - val_loss: 1.4371 - val_mse: 0.0102 - val_accuracy: 0.6560
Epoch 4/200
19/19 [==============================] - 20s 1s/step - loss: 0.1775 - mse: 0.0017 - accuracy: 0.9440 - val_loss: 1.4301 - val_mse: 0.0101 - val_accuracy: 0.6560
Epoch 5/200
19/19 [==============================] - 20s 1s/step - loss: 0.1601 - mse: 0.0016 - accuracy: 0.9449 - val_loss: 1.4230 - val_mse: 0.0101 - val_accuracy: 0.6560
Epoch 6/200
19/19 [==============================] - 20s 1s/step - loss: 0.1661 - mse: 0.0016 - accuracy: 0.9458 - val_loss: 1.4163 - v

In [ ]:
model.save('second_etap.hdf5')

In [ ]:
ls

sample_data/


#Для сдачи

In [ ]:
def get_data(train_gt, train_img_dir):
  import os
  import numpy as np
  from PIL import Image
  from tqdm import tqdm

  set_all_images = []
  answ = [] 
  for f in tqdm (sorted(os.listdir())):
    jpg = Image.open(f)
    gt = answ_files[f]
    answ.append(gt) 
    fixed_size_jpg = jpg.resize((197, 197))
    np_image = np.array(fixed_size_jpg)
    if(np.size(np_image.shape) == 2):
      k = np.dstack((np_image, np_image, np_image))
      set_all_images.append(k)
    else:
      set_all_images.append(np_image)
  set_all_images = np.array(set_all_images, dtype = np.flaot64)
  answ = np.array(answ, dtype = np.flaot64)
  return set_all_images, answ

def train_classifier(train_gt, train_img_dir, fast_train=True):
  import tensorflow as tf
  import numpy as np
  from keras import optimizers



  set_all_images, answ = get_data(train_gt, train_img_dir)
  #normalization
  set_all_images /= 255.0
  #корректное представление для answ
  answ_cat = np.array(tf.keras.utils.to_categorical(answ, num_classes=50))
  #архитектура
  base_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(197, 197, 3),
    include_top=False)
  
  base_model.trainable = False
  inputs = tf.keras.Input(shape=(197, 197, 3))
  x = base_model(inputs, training=False)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dense(512, activation='relu')(x)
  outputs =  tf.keras.layers.Dense(50, activation = 'softmax')(x)
  model = tf.keras.Model(inputs, outputs)
  #параметры_1
  model.compile(
    loss = 'categorical_crossentropy',
    optimizer='rmsprop',
    metrics = ['mse', 'accuracy'])
  #для генерации изображений
  datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,)
  #разделение на тренировачную и валид.выборку
  x_vld = set_all_images[::10]
  y_vld = answ_cat[::10]
  x_trn = np.array([])
  y_trn = np.array([])
  for i in range(1,10):
    if (i == 1):
      x_trn = np.copy(set_all_images[i::10])
      y_trn = np.copy(answ_cat[i::10])
    else:
      x_trn = np.concatenate((x_trn, set_all_images[i::10]), axis = 0)
      y_trn = np.concatenate((y_trn, answ_cat[i::10]), axis = 0)
  # тренирую выставленные мной слои
  model.fit(datagen.flow(x_trn,
          y_trn,
          batch_size=120),
          epochs=100,
          verbose=1,
          validation_data = (x_vld, y_vld),
          shuffle=True)
  #активирую слои базовой архитектуры
  for layer in base_model.layers[-19:]:
   layer.trainable = True
  #compile_2
  model.compile(
      loss = 'categorical_crossentropy',
      optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
      metrics = ['mse', 'accuracy']
  )
  #обучаю модель с последними активированными слоями
  model.fit(datagen.flow(x_trn,
        y_trn,
        batch_size=120),
        epochs=200,
        verbose=1,
        validation_data = (x_vld, y_vld),
        shuffle=True)
  return model

In [ ]:
def classify(model, test_img_dir):
  import os
  import numpy as np
  from PIL import Image
  from tqdm import tqdm

  answ_dict = dict()
  for f in tqdm (sorted(os.listdir(test_img_dir))):
    jpg = Image.open(f)
    fixed_size_jpg = jpg.resize((197, 197))
    np_image = np.array(fixed_size_jpg)
    if(np.size(np_image.shape) == 2):
      np_image = np.dstack((np_image, np_image, np_image))
    my_cur_prediction = model.predict(np_image)
    answ_dict[f] = my_cur_prediction
  return answ_dict

In [ ]:
!chmod +x build.sh

In [ ]:
!./build.sh

In [ ]:
!chmod +x run.py

In [ ]:
!./run.py public_tests

second_etap
2020-11-09 00:06:33.466185: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
optimizers
  0% 0/2500 [00:00<?, ?it/s]
2020-11-09 00:06:35.046397: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-09 00:06:35.088948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-09 00:06:35.089643: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-09 00:06:35.089697: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-09 00:06:35.0